In [4]:
import pandas as pd
dataset = pd.read_csv('mnist_train.csv')

print(len(dataset))

60000


A simple Neural Network Trained to recognize MNIST dataset.
\\

1. Forward Propogation: This takes your inputs (X) and spits out some output ($\hat{y}_i$). The way we will do this is we have 3 layers. We go from the input layer (784 neurons) $\to$ Hidden Layer 1 (10 neurons) $\to$ Output Layer (10 Neurons). \\


    When going from input layer to Hidden layer 1, we will have some W (weights) with shape (10 x 784) multiplied with x (784 x n). What this means is that for each of the 784 features taken from X, there will be some combination of those weights that determine how many of our 10 neurons are activated. Then we use some non-linear activation function (ReLu in this case), otherwise our entire NN will be nested linear functions which is linear. \\
    

    Note: Consider f(g(x)). If f and g are both linear functions like mx + b, then f(g(x)) is also of the form mx + b which will not capture the complexities of our questions. \\

    When going from Hidden Layer 1 to Output layer, we apply softmax which turns each output into a probability, with $\sum neurons$ = 1. \\

2. Back Prop: We are comparing the answer generated from forward prop to the real answer and go back and adjust the weights using gradient descent.

    We want to turn our answer vector into similar form as our output, which means we will use one-hot. This turns y which has value 8 into a vector [0, 0, 0, 0, 0, 0, 0, 0, 1, 0].T 

    For our first loss function from softmax, we will use the cross-entropy-loss function defined as \[ L = -\sum (y_i log(\hat{y}_i)) \]


    Then we apply gradient descent which moves these values from (learning rate) from where they are. Over ITERATIONS, these weights will become appropriately adjusted. 


3. Math:
    1. For our softmax function (dW2), consider $\frac{dL}{dW2} = \frac{dL}{d\hat{y}_i} \cdot \frac{d\hat{y}_i}{dz_2} \cdot \frac{dz_2}{dW2}$. The reason why we need all of these partials is because we can rewrite L in terms of the next terms. \\

        To solve this, lets look at \[ L := -\sum (y_i log(\hat{y}_i)) \] and break it down into its components. Here, $y_i$ is predetermined as 1 or 0, and we know that $\hat{y}_i$ is our softmax function from above, $\frac{e^{x_i}}{\sum e^{x_j}}$. Thus, our first partial comes out to $\frac{-y_i}{\hat{y}_i}$. \\

        For our next partial, (Softmax), we must consider this into 2 cases. On the top we have some $e^{x_i}$ term. So consider the case where our partial is with respect ti $x_i$ and then with respect to $x_j$ when $j \neq i$. \\

        In case 1: We will use simple calculus to solve this partial. Observe that \[
        \hat{y}_i := \frac{e^{x_i}}{\sum e^{x_j}}
        \]. Thus, we can simplify our expression down to $\hat{y}_i (1 - \hat{y}_i)$. \\

        Then, for the case when $i \neq j$, we will get $-\hat{y}_i \hat{y}_j$. \\

        Finally, for our last term $\frac{dz_2}{dW2}$, remember that Z2 = W2 $\cdot$ A1 (previous output layer) + $b_2$. So when taking the partial we are just left with $A1$. \\

        Putting this all together, we get the desired result. 

    \\

    Similarly for (b), just replace the last fraction with $dB2$. In fact, the partial comes out to be (1), which is even simpler! \\

    For our next term (dz1), Apply the same process. I'll write out the fraction decomposition and you can work through the math again (or re-use from above). $\frac{dL}{dZ1} = \frac{dL}{dA1}\frac{dA1}{dZ1}$. Then rewrite $\frac{dL}{dA1} := \frac{dL}{dZ2} \frac{dZ2}{dA1}$. We have already computed these values. The only term that is somewhat confusing is $\frac{dA1}{dz1}$. How do you take the derivative of a ReLu function? Well, since ReLu is 0 when x < 0, the derivative is 0 for x < 0. Similarly, since it is x when x $\geq$ 0, the derivative is 1 for all x > 0. Thus, we can use a trick and write $Z > 0$ since True gets assigned to 1. 


In [5]:
import numpy as np

data = np.array(dataset)
m, n = data.shape

print(data[:3])

print(m, n)

[[5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]]
60000 785


In [6]:

import random

random.shuffle(data)



In [7]:
validate = data[:1000]
train = data[1000:]


validate = validate.transpose()
y_validate = validate[0]
x_validate = validate[1:]
x_validate = x_validate / 255

print(y_validate.shape, x_validate.shape)

train = train.transpose()
y_train = train[0]
x_train = train[1:]
x_train = x_train / 255

print(y_train.shape, x_train.shape)



(1000,) (784, 1000)
(59000,) (784, 59000)


In [11]:
#forward propogation: 

#784 -(ReLu)-> 10 - Softmax -> 10 -> Output
def initiate_param():
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5

    return w1, b1, w2, b2
def Relu(z):
    return np.maximum(0, z)

def softmax(z):
    val = (np.exp(z)) / (np.sum(np.exp(z), axis = 0))
    return val
def forward_prop(W1, b1, W2, b2, X):
    z1 = np.dot(W1, X) + b1
    a1 = Relu(z1)
    z2 = np.dot(W2, a1) + b2
    a2 = softmax(z2)

    return (z1, a1, z2, a2)



#Backwards Prop
def one_hot(Y):
    one_hot_y = np.zeros((10, Y.size))
    one_hot_y[Y, np.arange(Y.size)] = 1
    
    return one_hot_y

def deriv_ReLu(Z):
    return Z > 0

def backprop(Z1, A1, Z2, A2, W2, X, Y):
    m = len(Y)
    one_hot_y = one_hot(Y)
    dZ2 = A2 - one_hot_y
    dW2 = 1 / m * np.dot(dZ2, A1.transpose())
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = np.dot(W2.transpose(), dZ2) * deriv_ReLu(Z1)
    dW1 = 1 / m * np.dot(dZ1, X.transpose())
    db1 = 1 / m * np.sum(dZ1)

    return db1, dW1, db2, dW2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1 
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    w1, b1, w2, b2 = initiate_param()
    for i in range(iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, X)
        db1, dw1, db2, dw2 = backprop(z1, a1, z2, a2, w2, X, Y)

        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        # print("This is new weights", w1, b1, w2,)
        if i % 100 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(a2)
            print(get_accuracy(predictions, Y))
    return w1, b1, w2, b2

    
    

In [12]:
w1, b1, w2, b2 = gradient_descent(x_train, y_train, 1000, 0.1)

Iteration:  0
0.10191525423728813
Iteration:  100
0.5928983050847457
Iteration:  200
0.7514237288135593
Iteration:  300
0.8073898305084746
Iteration:  400
0.834406779661017
Iteration:  500
0.8510677966101695
Iteration:  600
0.8618813559322034
Iteration:  700
0.8697966101694915
Iteration:  800
0.8760677966101695
Iteration:  900
0.8807966101694915


In [13]:
#Now comprae against the training data
_,_,_,a2 = forward_prop(w1, b1, w2, b2, x_validate)
predictions = get_predictions(a2)
acc = get_accuracy(predictions, y_validate)
print(acc)

0.894
